In [1]:
mnnmfrom numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

##### move to spacy and do lematization later

In [3]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)


In [4]:

df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

In [5]:
dataset = pd.concat([df1, df2], ignore_index=True)

In [6]:
len(dataset)

490

In [8]:
# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 490):
    review = re.sub('[^a-zA-Z]', ' ', dataset['desc'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

In [9]:
print(max_len)

114


In [10]:
all_words = [j for sub in all_words for j in sub] 

In [11]:
unique_words = set(all_words)
print(len(unique_words))
vocab_length = len(unique_words) + 10
vocab_length

753


763

In [12]:
dataset.head()

,class,desc,inc_num,short_des
0,P,"y2684992. good afternoon,\r\n\r\nthis customer...",INC0809260,y2684992
1,R,delivery pass needs removing on staff accounts...,INC0808628,delivery pass needs removing on staff accounts...
2,R,delivery pass needs removing on staff accounts...,INC0807253,delivery pass needs removing on staff accounts...
3,P,customer being charged for delivery. customer:...,INC0806836,customer being charged for delivery
4,P,website/web shop issues. short description:tam...,INC0805994,website/web shop issues


In [120]:
dataset['inc_num'][49]

'INC0758972'

In [13]:
len(corpus[49])

684

In [162]:
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer().fit(padded_sent)
# padded_sentences = transformer.transform(padded_sent)

In [17]:
print(padded_sentences[0])

NameError: name 'padded_sentences' is not defined

In [15]:
from keras.preprocessing.text import Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

In [16]:
one_hot_results = word_tokenizer.texts_to_matrix(corpus, mode='binary')
len(one_hot_results[3])

754

In [18]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
len(embedded_sentences[49])

114

In [19]:
padded_sentences = pad_sequences(embedded_sentences, max_len, padding='post')

In [20]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open(r'C:\Users\AVNEET\Desktop\glove.6B.100d.txt', encoding="utf8")

In [21]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [22]:
embedding_matrix = zeros((vocab_length, 100))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [34]:
len(embedding_matrix[752])

100

### Tfidf Vectorizer

In [30]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()

# X_train_tfidf = vectorizer.fit_transform(corpus) # remember to use the original X_train set
# X_train_tfidf.shape

(488, 727)

In [33]:
# X_train_tfidf

<488x727 sparse matrix of type '<class 'numpy.float64'>'
	with 12476 stored elements in Compressed Sparse Row format>

### Deep Learning

In [24]:
model = Sequential()
model.add(Embedding(vocab_length,100,weights=[embedding_matrix], input_length=max_len,trainable=False))
model.add(Flatten())
# model.add(Dense(vocab_length, input_dim=30, activation='softmax'))
# model.add(Dense(vocab_length, input_dim=30, activation='softmax'))
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [45]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 114, 100)          76200     
_________________________________________________________________
flatten_1 (Flatten)          (None, 11400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22802     
Total params: 99,002
Trainable params: 22,802
Non-trainable params: 76,200
_________________________________________________________________
None


In [23]:
cldict = {'P':1 , 'R':0}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

In [24]:
from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y = to_categorical(y)
y.shape

(488, 2)

In [25]:
X = padded_sentences

In [26]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)


In [49]:
model.fit(X_train, y_train, epochs=300, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [133]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

74/74 [==============================] - 0s 122us/step
Accuracy: 98.648649


In [58]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
k_fold = KFold(n_splits=20, shuffle=True, random_state=7)
acc_lst=[]
for tr, test in k_fold.split(X, y):
    loss, accur = model.evaluate(X[test], y[test], verbose=0)   
    accur=accur*100
    print(accur)
    acc_lst.append(accur)
np.mean(acc_lst)

95.99999785423279
100.0
100.0
100.0
100.0
100.0
100.0
95.99999785423279
100.0
100.0
95.83333134651184
95.83333134651184
95.83333134651184
100.0
100.0
95.83333134651184
91.66666865348816
100.0
91.66666865348816
95.83333134651184


97.72499948740005

In [48]:
model.predict(X_test[:5])

array([[9.4521551e-05, 9.9990547e-01],
       [4.0384810e-04, 9.9959618e-01],
       [9.9384270e-05, 9.9990058e-01],
       [9.9999154e-01, 8.4906414e-06],
       [8.2660872e-01, 1.7339128e-01]], dtype=float32)

In [52]:
model.save('.h5')

In [56]:
from keras.models import load_model
model = load_model('P&PGloVe.h5')

In [55]:
#strin = 'please add the delivery subscription'
# pad = re.sub('[^a-zA-Z]', ' ', pad)
vocab_length = 762
max_len = 114
pad = strin.lower()
pad = pad.split()
pad = ' '.join(pad)
pad = nlp(pad)
pad = [word.text for word in pad]
pad = ' '.join(pad)
emb = [one_hot(pad, vocab_length)]
pady = pad_sequences(emb, max_len, padding='post')

x = model.predict_classes(pady)
print(x)
if x[0] == 1:
    print('P', model.predict(pady))
    
else: 
        print('R', model.predict(pady))
    

[1]
P [[0.01869247 0.9813075 ]]


In [53]:
strin = 'Hi the delivery passes need removing from these staff accounts This is still being to Staff accounts for £0 01 please can you remove this Thank You'

In [184]:
from sklearn import metrics

# Create a prediction set:
predictions = model.predict_classes(X)

# Print a confusion matrix
print(metrics.confusion_matrix(y.argmax(axis=1),predictions))

[[182   8]
 [  3 295]]


In [59]:
y_test.argmax(axis=1)

array([0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1], dtype=int64)

In [120]:
3

predicted class:[1] [[5.175987e-04 9.994824e-01]] Index is : 5
predicted class:[1] [[0.16320924 0.83679074]] Index is : 10
predicted class:[1] [[0.39117888 0.60882115]] Index is : 14
predicted class:[1] [[0.01174793 0.9882521 ]] Index is : 30
predicted class:[1] [[0.37375617 0.6262438 ]] Index is : 36
predicted class:[1] [[9.4522002e-05 9.9990547e-01]] Index is : 52
predicted class:[0] [[0.94681954 0.05318044]] Index is : 63
predicted class:[1] [[0.18990098 0.81009907]] Index is : 74
predicted class:[1] [[0.06214963 0.93785036]] Index is : 77


In [119]:
s=0
for j in range(0,488):
    for k in range(0,9):
        if (np.array_equal(arr[k], padded_sentences[j])):
            print(k, '**',corpus[j],'****', j , dataset['inc_num'][j])
print(s)   

1 ** delivery pass needs removing on orders supra dba req please can the delivery pass be removed from the attached staff accounts thank you **** 9 INC0793103
4 ** annual delivery pass cancelled please can the annual delivery pass service be removed from customers account a thanks **** 11 INC0791140
5 ** a a hi this is a new staff account it is showing as free delivery can this be removed please thanks chris **** 117 INC0690294
8 ** delivery pass needs removing on staff accounts supra dba req please can you remove the delivery pass from the attached staff accounts thank you **** 124 INC0684243
7 ** delivery subscription needs removing u please can you remove the delivery subscription from account thank you **** 193 INC0622277
3 ** customer wishes to cancel annual delivery t please can you remove the delivery subscription service from this account thanks **** 195 INC0622190
6 ** annual delivery pass not working please apply delivery pass to account t purchased thanks **** 211 INC0595802

In [60]:
(X_test)

numpy.ndarray

In [107]:
if (arr[0] == padded_sentences[0]).all:
    print('x')

x


In [110]:
len(arr[0])

114

In [111]:
len(padded_sentences[0])

114

In [113]:
padded_sentences[0].shape

(114,)

In [114]:
np.array_equal(arr[0], padded_sentences[0])

False